In [1]:
import torch
from torch import nn
from torch.nn import functional as F

## Create Layers and Basic network

In [2]:
net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
X = torch.rand(2, 20)
net(X)

tensor([[ 0.1534, -0.1974, -0.1253,  0.2826, -0.0359, -0.1332, -0.1085, -0.1724,
         -0.0973,  0.1372],
        [ 0.0994, -0.0959, -0.0798,  0.3519, -0.0684, -0.2144, -0.1779, -0.1720,
         -0.1881,  0.3157]], grad_fn=<AddmmBackward>)

In [3]:
class MLP(nn.Module):
  def __init__(self):
    super().__init__()
    self.hidden = nn.Linear(20, 256)
    self.out = nn.Linear(256, 10)

  def forward(self, x):
    return self.out(F.relu(self.hidden(x)))

net = MLP()
net(X)

tensor([[ 0.1584, -0.0616, -0.0930, -0.1899,  0.1299,  0.1143,  0.2299,  0.0235,
         -0.0137,  0.1715],
        [ 0.2392, -0.1084,  0.1165, -0.1745,  0.1591, -0.0368,  0.2296, -0.0040,
         -0.0923,  0.2156]], grad_fn=<AddmmBackward>)

## Make Sequential Block

In [6]:
class MySequential(nn.Module):
  def __init__(self, *args):
    super().__init__()
    for idx, module  in enumerate(args):
      self._modules[str(idx)] = module

  def forward(self, X):
    for block in self._modules.values():
      X = block(X)
    return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.0132, -0.0630,  0.0739, -0.0601, -0.1561,  0.0772,  0.0549,  0.1660,
         -0.0271,  0.1281],
        [ 0.0070, -0.0787,  0.1284, -0.0910, -0.0709,  0.1599, -0.0185,  0.1928,
         -0.1674,  0.0612]], grad_fn=<AddmmBackward>)

In [7]:
class FixedHiddenMLP(nn.Module):
  def __init__(self):
    super().__init__()
    self.rand_weight = torch.rand((20, 20), requires_grad=False)
    self.linear = nn.Linear(20, 20)

  def forward(self, X):
    X = self.linear(X)
    X = F.relu(torch.mm(X, self.rand_weight) + 1)
    X = self.linear(X)
    while X.abs().sum() > 1:
      X /= 2
    return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(0.2313, grad_fn=<SumBackward0>)

In [9]:
class NestMLP(nn.Module):
  def __init__(self):
      super().__init__()
      self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                nn.Linear(64, 32), nn.ReLU())
      self.linear = nn.Linear(32, 16)

  def forward(self, X):
      return self.linear(self.net(X))

# 64 -> 32 -> 16
chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.0960, grad_fn=<SumBackward0>)